In [39]:
#Import Modules

#GPyOpt - Cases are important, for some reason
import GPyOpt
from GPyOpt.methods import BayesianOptimization

#numpy
import numpy as np
from numpy.random import multivariate_normal #For later example

import pandas as pd

#Plotting tools
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from numpy.random import multivariate_normal

# --- Load GPyOpt
from GPyOpt.methods import BayesianOptimization
%pylab inline
import GPyOpt
import GPy
import numpy as np
import pickle

Populating the interactive namespace from numpy and matplotlib


In [42]:
def f(x):
    """
    x is a 4D vector.
    Function which will send alpha_1, alpha_2, alpha_3 and alpha_4 
    to the actual model and will get the dice coefficient in return.
    """
    alpha_1 = x[:, 0][0]
    alpha_2 = x[:, 1][0]
    alpha_3 = x[:, 2][0]
    alpha_4 = x[:, 3][0]
    print("--------",alpha_1," ",alpha_2," ",alpha_3," ",alpha_4)
    print(alpha_1+alpha_2+alpha_3+alpha_4)
    # Here we will send the alphas to the actual model and in return 
    # we will recieve the dice coefficient to optimise, since this is
    # a maximization problem, we return the -ve of objective function
    # to be maximized
    dice_coef = alpha_1 + alpha_2 + alpha_3 + alpha_4
    return -dice_coef

domain = [{'name': 'alpha_1', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_2', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_3', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1},
          {'name': 'alpha_4', 'type': 'continuous', 'domain': (0,1), 'dimensionality':1}]

constraints = [{'name': 'constr_1', 'constraint': '0.9999 - x[:,0] - x[:,1] - x[:,2] - x[:,3]'},
               {'name': 'constr_1', 'constraint': '-1.00001 + x[:,0] + x[:,1] + x[:,2] + x[:,3]'}]

In [43]:
maxiter = 2

kernel = GPy.kern.Matern52(input_dim=4, ARD=True, variance=1, lengthscale=[1,1,1,1]);

myBopt_4d = GPyOpt.methods.BayesianOptimization(f, domain=domain, constraints = constraints,kernel=kernel,
                                                acquisition_type ='EI', model_type='GP', verbosity=True,
                                                acquisition_optimizer_type='lbfgs', cost_withGradients=None,
                                                exact_feval=True)

myBopt_4d.run_optimization(max_iter = maxiter, verbosity=True)
print("="*20)
print("Value of (x,y) that minimises the objective:"+str(myBopt_4d.x_opt))    
print("Minimum value of the objective: "+str(myBopt_4d.fx_opt))     
print("="*20)
#myBopt_4d.plot_acquisition()

-------- 0.4504489083409762   0.2203649190311513   0.24142062972621636   0.08776164449513724
0.9999961015934811
-------- 0.24486872951636418   0.38571899855611813   0.11917063687439189   0.2502213845651453
0.9999797495120195
-------- 0.3909029421326983   0.5341746401740595   0.013502306268339592   0.06138035555307286
0.9999602441281702
-------- 0.18199088232975147   0.31714799632895774   0.4987861257634475   0.002079975281623958
1.0000049797037807
-------- 0.04209037314131103   0.011597466231884401   0.3332378291901311   0.6130521680948896
0.9999778366582162


KeyboardInterrupt: 